# Genetic Algorithm Assignment
30% of the overall grade for this module

Marks indciated in sections below are based on percentage of marks allocated for this module

In this assignment you must choose a problem, and attempt to use the Genetic Alogrithm that we developed in class to solve this problem.





## The Problem         **(~30%)**

*   Description of the problem





---

The problem at hand is a classic Sudoku puzzle, represented as a 9x9 grid. Each row, column, and 3x3 subgrid must contain the numbers 1 through 9 without repetition. The goal is to find a solution that satisfies all Sudoku rules and fills in the empty cells of any given Sudoku puzzle.

---



*   Discussion of the suitablity of Genetic Algorithms


---


Genetic Algorithms are well-suited for solving optimization problems, and Sudoku is one such problem. GAs emulate the process of natural selection and evolution to search for the best possible solution. In the context of Sudoku, a potential solution can be represented as a chromosome, consisting of 9 genes, where each gene represents a row in the grid. The algorithm evolves a population of candidate solutions through processes like crossover and mutation, gradually improving the fitness of the individuals until an optimal or near-optimal solution is found. Given the discrete and combinatorial nature of Sudoku, where valid solutions need to satisfy a set of constraints, Genetic Algorithms provide an effective approach for exploring the solution space and finding satisfactory solutions.

---



*   Complexity of the problem  (Overall marks allocated based on ..)

---

The complexity of a Sudoku problem lies in its combinatorial nature, where a seemingly simple 9x9 grid conceals intricate patterns and interdependencies. While Sudoku puzzles typically start with a partially filled grid, the challenge emerges from the need to complete the puzzle by adhering to strict rules: each row, column, and 3x3 subgrid must contain the numbers 1 through 9 without repetition. The constraint satisfaction problem inherent in Sudoku makes it NP-complete, meaning there is no known polynomial-time algorithm to solve all instances efficiently. As one progresses in solving a Sudoku puzzle, the interlocking constraints introduce a cascading effect, where filling one cell influences multiple others. The logical deductions required for successful solving often demand a combination of pattern recognition, elimination strategies, and occasionally trial-and-error, adding to the puzzle's complexity. Sudoku's appeal lies in its ability to engage both novice and expert puzzle solvers, with varying degrees of difficulty stemming from the intricate interplay of constraints within the grid.

---

# The problem and the cost function   **(~20%)**

In [2]:
import numpy as np
from copy import deepcopy

In [3]:
def cost_function(chromosome):
    cost = 0

    for col in range(9):
        unique_digits = set()
        for row in chromosome:
            digit = row[col]
            if digit in unique_digits:
                cost += 1
            unique_digits.add(digit)

    for i in range(0, 9, 3):
        for j in range(0, 9, 3):
            unique_digits = set()
            for x in range(3):
                for y in range(3):
                    digit = chromosome[i + x][j + y]
                    if digit in unique_digits:
                        cost += 1
                    unique_digits.add(digit)

    return cost

---

***Cost Function:***

This cost function is designed for evaluating the validity of a Sudoku puzzle represented by a 9x9 or grid, referred to as a "chromosome" here. The function iterates through each column and each 3x3 subgrid in the Sudoku puzzle, keeping track of the number of repeated digits. In the first loop, it examines each column and increments the cost for every repeated digit within the same column. In the second set of nested loops, it focuses on each 3x3 subgrid within the puzzle and tallies the cost for repeated digits within each subgrid. The cost is incremented whenever a digit is found to be repeated. The overall cost reflects the number of violations of the Sudoku rules, where a valid Sudoku puzzle should have each digit from 1 to 9 occurring exactly once in each row, column, and 3x3 subgrid. The goal is to minimize this cost to ensure the puzzle adheres to the rules of Sudoku.

---

In [178]:
class problem:
    def __init__(self, puzzle):
        self.puzzle = puzzle
        self.cost_function = cost_function

---

***The Problem:***

The problem class represents a Sudoku puzzle as a 9x9 grid, with some cells initially filled and others left blank. The puzzle configuration is stored in the puzzle attribute. The class includes a cost_function attribute, which refers to a function that evaluates the puzzle's validity based on Sudoku rules by calculating the number of digit repetitions.

---

# The Individual **(~30%)**


*   Chromosone
*   Crossover
*   Mutation



## Discussion and justification on the approaches taken for the above

---

***Chromosome***

The individual class represents a candidate solution in a genetic algorithm designed to solve a Sudoku puzzle. The chromosome attribute of each individual is a representation of a possible solution, organised as a 9x9 grid of genes. The genes are generated using the make_gene function, which shuffles the numbers 1 to 9 and ensures that each row has unique values. This approach is taken in order to avoid duplication in rows, simplifying the process of checking for repeating numbers during the cost function calculation.

***Crossover***

The crossover method implements the genetic operation of crossover (or recombination). It takes another individual (parent2) as input and creates two offspring by randomly selecting genes from both parents. This mimics the natural process of genetic recombination, promoting diversity among solutions. The use of a binary random variable (x) determines which genes are inherited from each parent. This introduces variability and encourages exploration of the solution space.

***Mutation***

The mutate method introduces genetic diversity by randomly changing genes with a specified mutation rate. For each gene in the individual's chromosome, there is a probability that it will be mutated. If the mutation occurs, the gene is replaced with a new one generated by the make_gene function. The mutation is performed on a per-gene basis, and the mutation rate parameter allows for fine-tuning the balance between exploration and exploitation in the algorithm.

In summary, the design choices in the chromosome representation, crossover, and mutation functions are well-justified. The use of a unique gene creation process ensures that rows in the puzzle do not contain repeated numbers, simplifying the constraint satisfaction. The crossover and mutation operations contribute to the genetic algorithm's ability to explore diverse solutions.

---



In [127]:
def make_gene(initial=None):
    if initial is None:
        initial = [0] * 9

    mapp = {}
    gene = list(range(1, 10))
    np.random.shuffle(gene)

    for i in range(9):
        mapp[gene[i]] = i

    for i in range(9):
        if initial[i] != 0 and gene[i] != initial[i]:
            temp = gene[i], gene[mapp[initial[i]]]
            gene[mapp[initial[i]]], gene[i] = temp
            mapp[initial[i]], mapp[temp[0]] = i, mapp[initial[i]]

    return gene

In [128]:
def make_chromosome(initial=None):
    if initial is None:
        initial = [[0] * 9] * 9
    chromosome = []
    for i in range(9):
        chromosome.append(make_gene(initial[i]))
    return chromosome

In [129]:
class individual:
  def __init__(self, prob):
    self.chromosome = make_chromosome(prob.puzzle)
    self.cost = prob.cost_function(self.chromosome)

  def crossover(self, parent2):
    child_1 = deepcopy(self)
    child_2 = deepcopy(parent2)
    for i in range(9):
        x = np.random.randint(0, 2)
        if x == 1:
            child_1.chromosome[i] = self.chromosome[i]
            child_2.chromosome[i] = parent2.chromosome[i]
        elif x == 0:
            child_2.chromosome[i] = self.chromosome[i]
            child_1.chromosome[i] = parent2.chromosome[i]

    return child_1, child_2

  def mutate(self, rate_of_gene_mutation, prob):
    for i in range(9):
        if np.random.uniform() < rate_of_gene_mutation:
            self.chromosome[i] = make_gene(prob.puzzle[i])
    return self.chromosome

---

*** FEW BASIC TESTS ***

---

In [179]:
easy_puzzle = [
    [3,4,2,0,8,0,0,0,0],
    [5,0,0,9,0,0,0,2,0],
    [0,9,0,0,0,4,3,8,0],
    [0,2,0,3,0,5,1,0,0],
    [0,5,0,7,0,6,0,4,0],
    [0,7,0,0,9,1,6,5,2],
    [6,0,0,0,7,9,2,3,1],
    [7,0,0,0,6,0,8,0,0],
    [2,0,0,5,3,0,4,0,0]
]

In [180]:
easy_problem = problem(easy_puzzle)

In [181]:
ind1 = individual(easy_problem)

In [182]:
ind2 = individual(easy_problem)

In [183]:
ind1.chromosome

[[3, 4, 2, 7, 8, 1, 6, 9, 5],
 [5, 6, 7, 9, 8, 4, 1, 2, 3],
 [5, 9, 1, 7, 6, 4, 3, 8, 2],
 [6, 2, 9, 3, 7, 5, 1, 8, 4],
 [9, 5, 3, 7, 1, 6, 8, 4, 2],
 [8, 7, 4, 3, 9, 1, 6, 5, 2],
 [6, 4, 8, 5, 7, 9, 2, 3, 1],
 [7, 1, 4, 5, 6, 3, 8, 2, 9],
 [2, 9, 6, 5, 3, 1, 4, 7, 8]]

In [184]:
ind2.chromosome

[[3, 4, 2, 5, 8, 1, 9, 6, 7],
 [5, 3, 8, 9, 4, 7, 1, 2, 6],
 [7, 9, 1, 2, 6, 4, 3, 8, 5],
 [7, 2, 9, 3, 4, 5, 1, 8, 6],
 [1, 5, 2, 7, 9, 6, 3, 4, 8],
 [3, 7, 4, 8, 9, 1, 6, 5, 2],
 [6, 8, 4, 5, 7, 9, 2, 3, 1],
 [7, 2, 4, 5, 6, 1, 8, 9, 3],
 [2, 6, 8, 5, 3, 9, 4, 7, 1]]

In [185]:
ind1.cost

43

In [186]:
ind2.cost

39

In [187]:
ch1, ch2 = ind1.crossover(ind2)

In [188]:
ch1.chromosome

[[3, 4, 2, 7, 8, 1, 6, 9, 5],
 [5, 6, 7, 9, 8, 4, 1, 2, 3],
 [7, 9, 1, 2, 6, 4, 3, 8, 5],
 [6, 2, 9, 3, 7, 5, 1, 8, 4],
 [9, 5, 3, 7, 1, 6, 8, 4, 2],
 [8, 7, 4, 3, 9, 1, 6, 5, 2],
 [6, 4, 8, 5, 7, 9, 2, 3, 1],
 [7, 1, 4, 5, 6, 3, 8, 2, 9],
 [2, 6, 8, 5, 3, 9, 4, 7, 1]]

In [189]:
ch1.mutate(.25, easy_problem)

[[3, 4, 2, 7, 8, 1, 6, 9, 5],
 [5, 6, 7, 9, 8, 4, 1, 2, 3],
 [7, 9, 1, 2, 6, 4, 3, 8, 5],
 [6, 2, 9, 3, 7, 5, 1, 8, 4],
 [2, 5, 8, 7, 9, 6, 3, 4, 1],
 [3, 7, 8, 4, 9, 1, 6, 5, 2],
 [6, 8, 5, 4, 7, 9, 2, 3, 1],
 [7, 3, 5, 2, 6, 4, 8, 9, 1],
 [2, 6, 8, 5, 3, 9, 4, 7, 1]]

## Running the algorithm  **(~10%)**

*   Parameter choices
*   Modifications (if any) to run_genetic
*   Rationale for the above



---
***Parameters***

The population size determines the number of potential solutions in each generation, influencing the diversity of the population and the algorithm's ability to explore different solution spaces. A larger population often allows for a more thorough exploration but comes with increased computational costs. The child rate per generation dictates how many offspring are generated from each pair of parents, impacting the balance between exploration and exploitation of promising solutions. A higher child rate can lead to a more rapid convergence, but it might also increase the risk of premature convergence to suboptimal solutions. The rate of gene mutation introduces randomness, allowing the algorithm to explore new possibilities and escape local optima. Lastly, the number of iterations represents the maximum number of generations the algorithm will run, acting as a termination criterion to prevent indefinite execution.

***Run_Genetics***

I maintained the structure of the genetic algorithm in the run_genetic function while simplifying it to suit my specific needs. I removed unnecessary variables such as max_number_of_generations, explore_crossover, and mutate_range, streamlining the code for clarity and focusing on the essential parameters. The core functionality, including the population initialization, generational iteration, and selection of the best solution, remains intact. By eliminating redundant variables, I aimed to create a more straightforward and tailored genetic algorithm implementation, emphasizing the parameters essential to my specific problem.



---



In [491]:
class parameters:
  def __init__(self):
    self.population = 500
    self.child_rate_per_generation = 1
    self.rate_of_gene_mutation = .2
    self.iterations = 100

In [492]:
params = parameters()

In [493]:
def choose_indices_from(list_size):
  index_1 = np.random.randint(list_size)
  index_2 = np.random.randint(list_size)

  if index_1 == index_2:
    return choose_indices_from(list_size)

  return index_1, index_2

In [494]:
def run_genetic(prob, params):
     # Read in important info
    cost_function = prob.cost_function
    number_in_population = params.population
    number_of_children_per_generation = params.child_rate_per_generation * number_in_population
    mutate_rate = params.rate_of_gene_mutation
    max_iterations = params.iterations

    # Generate initial population
    population = []

    # Placeholder for best solution
    best_solution = individual(prob)
    best_solution.cost = np.infty

    # Populate the population
    for i in range(number_in_population):
        new_individual = individual(prob)
        population.append(new_individual)

    # Find best individual
    if new_individual.cost < best_solution.cost:
        best_solution = deepcopy(new_individual)

    # Generational Iteration
    _iteration = 0
    while best_solution.cost > 0  and _iteration < max_iterations:
        # Create children
        children = []

        while len(children) < number_of_children_per_generation:
            parent1_index, parent2_index = choose_indices_from(len(population))
            parent1 = population[parent1_index]
            parent2 = population[parent2_index]

            # Get children
            c1, c2 = parent1.crossover(parent2)
            c1.mutate(mutate_rate, prob)
            c2.mutate(mutate_rate, prob)

            # Calculate cost
            c1.cost = cost_function(c1.chromosome)
            c2.cost = cost_function(c2.chromosome)

            # Add children to list of children
            children.append(c1)
            children.append(c2)

        # All children created.

        # Add children to population
        population += children

        # Sort the population
        population = sorted(population, key=lambda x: x.cost)

        # Cull the population
        population = population[:number_in_population]

        # Adjust the best solution
        if population[0].cost < best_solution.cost:
            best_solution = deepcopy(population[0])

        print("Best Solution for iteration " + str(_iteration) + " has cost of " + str(best_solution.cost))

        _iteration += 1

    return best_solution

In [495]:
bs_easy = run_genetic(easy_problem, params)

Best Solution for iteration 0 has cost of 27
Best Solution for iteration 1 has cost of 27
Best Solution for iteration 2 has cost of 26
Best Solution for iteration 3 has cost of 24
Best Solution for iteration 4 has cost of 23
Best Solution for iteration 5 has cost of 23
Best Solution for iteration 6 has cost of 17
Best Solution for iteration 7 has cost of 17
Best Solution for iteration 8 has cost of 17
Best Solution for iteration 9 has cost of 17
Best Solution for iteration 10 has cost of 17
Best Solution for iteration 11 has cost of 15
Best Solution for iteration 12 has cost of 15
Best Solution for iteration 13 has cost of 15
Best Solution for iteration 14 has cost of 15
Best Solution for iteration 15 has cost of 15
Best Solution for iteration 16 has cost of 12
Best Solution for iteration 17 has cost of 12
Best Solution for iteration 18 has cost of 12
Best Solution for iteration 19 has cost of 10
Best Solution for iteration 20 has cost of 10
Best Solution for iteration 21 has cost of 1

---

Easy puzzle to approx 4 seconds to solve

***Solution below***

---

In [195]:
bs_easy.chromosome

[[3, 4, 2, 6, 8, 7, 5, 1, 9],
 [5, 6, 8, 9, 1, 3, 7, 2, 4],
 [1, 9, 7, 2, 5, 4, 3, 8, 6],
 [9, 2, 6, 3, 4, 5, 1, 7, 8],
 [8, 5, 1, 7, 2, 6, 9, 4, 3],
 [4, 7, 3, 8, 9, 1, 6, 5, 2],
 [6, 8, 5, 4, 7, 9, 2, 3, 1],
 [7, 3, 4, 1, 6, 2, 8, 9, 5],
 [2, 1, 9, 5, 3, 8, 4, 6, 7]]

---

***MEDIUM PUZZLE***

Using a medium puzzle from Sudoku.com, we will try to solve it using our GA.

---

In [547]:
medium_puzzle = [
    [0,0,5,0,7,0,2,6,0],
    [7,0,3,0,0,0,0,8,1],
    [9,0,0,0,6,5,0,0,0],
    [8,3,0,7,5,0,0,4,0],
    [0,7,0,4,0,9,1,0,0],
    [5,0,0,3,0,0,0,0,0],
    [0,0,0,2,0,0,6,9,0],
    [0,0,6,0,4,0,0,0,2],
    [0,0,0,0,0,0,0,5,7]
]

In [548]:
class mediumparameters:
  def __init__(self):
    self.population = 100
    self.child_rate_per_generation = 10
    self.rate_of_gene_mutation = 0.1
    self.iterations = 1500

In [549]:
medium_params = mediumparameters()

In [550]:
medium_problem = problem(medium_puzzle)

In [552]:
bs_medium = run_genetic(medium_problem, medium_params)

Best Solution for iteration 0 has cost of 35
Best Solution for iteration 1 has cost of 30
Best Solution for iteration 2 has cost of 27
Best Solution for iteration 3 has cost of 25
Best Solution for iteration 4 has cost of 20
Best Solution for iteration 5 has cost of 20
Best Solution for iteration 6 has cost of 18
Best Solution for iteration 7 has cost of 18
Best Solution for iteration 8 has cost of 15
Best Solution for iteration 9 has cost of 13
Best Solution for iteration 10 has cost of 13
Best Solution for iteration 11 has cost of 9
Best Solution for iteration 12 has cost of 8
Best Solution for iteration 13 has cost of 7
Best Solution for iteration 14 has cost of 6
Best Solution for iteration 15 has cost of 6
Best Solution for iteration 16 has cost of 5
Best Solution for iteration 17 has cost of 5
Best Solution for iteration 18 has cost of 5
Best Solution for iteration 19 has cost of 5
Best Solution for iteration 20 has cost of 5
Best Solution for iteration 21 has cost of 5
Best Solu

---

Medium puzzle was solved in approx. 9 seconds

***Solution Below***

---

In [553]:
bs_medium.chromosome

[[4, 1, 5, 8, 7, 3, 2, 6, 9],
 [7, 6, 3, 9, 2, 4, 5, 8, 1],
 [9, 2, 8, 1, 6, 5, 4, 7, 3],
 [8, 3, 1, 7, 5, 2, 9, 4, 6],
 [6, 7, 2, 4, 8, 9, 1, 3, 5],
 [5, 4, 9, 3, 1, 6, 7, 2, 8],
 [1, 5, 7, 2, 3, 8, 6, 9, 4],
 [3, 9, 6, 5, 4, 7, 8, 1, 2],
 [2, 8, 4, 6, 9, 1, 3, 5, 7]]

---

***HARD PUZZLE***

Using a hard puzzle from Sudoku.com, we will try to solve it using our GA.

---

In [665]:
hard_puzzle = [
    [2,0,0,9,0,0,6,0,0],
    [0,0,0,5,0,0,8,0,0],
    [8,0,6,0,7,0,1,0,0],
    [4,0,0,0,0,1,2,0,0],
    [0,0,0,0,0,0,0,0,0],
    [6,0,0,7,0,0,0,0,9],
    [0,8,5,0,0,0,0,0,0],
    [0,4,0,0,5,0,0,0,2],
    [7,0,0,4,0,0,0,0,3]
]

In [785]:
class hardparameters:
  def __init__(self):
    self.population = 500
    self.child_rate_per_generation = 10
    self.rate_of_gene_mutation = .1
    self.iterations = 600

In [786]:
hard_problem = problem(hard_puzzle)

In [787]:
hard_params = hardparameters()

In [788]:
bs_hard = run_genetic(hard_problem, hard_params)

Best Solution for iteration 0 has cost of 31
Best Solution for iteration 1 has cost of 30
Best Solution for iteration 2 has cost of 27
Best Solution for iteration 3 has cost of 27
Best Solution for iteration 4 has cost of 23
Best Solution for iteration 5 has cost of 21
Best Solution for iteration 6 has cost of 21
Best Solution for iteration 7 has cost of 18
Best Solution for iteration 8 has cost of 14
Best Solution for iteration 9 has cost of 14
Best Solution for iteration 10 has cost of 14
Best Solution for iteration 11 has cost of 14
Best Solution for iteration 12 has cost of 13
Best Solution for iteration 13 has cost of 12
Best Solution for iteration 14 has cost of 12
Best Solution for iteration 15 has cost of 12
Best Solution for iteration 16 has cost of 11
Best Solution for iteration 17 has cost of 11
Best Solution for iteration 18 has cost of 10
Best Solution for iteration 19 has cost of 10
Best Solution for iteration 20 has cost of 9
Best Solution for iteration 21 has cost of 9


---
Hard Puzzle unsolved after 9 minutes and 600 iterations.

***Closest Solution Below***

---

In [791]:
bs_hard.chromosome

[[2, 5, 7, 9, 1, 8, 6, 3, 4],
 [1, 9, 4, 5, 6, 3, 8, 2, 7],
 [8, 3, 6, 2, 7, 4, 1, 5, 9],
 [4, 7, 9, 8, 3, 1, 2, 6, 5],
 [1, 2, 3, 6, 9, 5, 4, 7, 8],
 [6, 5, 8, 7, 4, 2, 3, 1, 9],
 [3, 8, 5, 1, 2, 7, 9, 4, 6],
 [9, 4, 1, 3, 5, 6, 7, 8, 2],
 [7, 6, 2, 4, 8, 9, 5, 1, 3]]

---

***EXPERT PUZZLE***

Using a expert puzzle from Sudoku.com, we will try to solve it using our GA.

---

In [792]:
expert_puzzle = [
    [6,0,0,2,0,0,1,0,7],
    [9,0,0,0,0,7,0,0,2],
    [0,2,0,0,0,0,0,0,0],
    [0,0,1,3,4,0,0,0,0],
    [0,0,0,8,0,0,0,0,0],
    [3,4,0,0,0,0,0,5,0],
    [0,0,8,0,6,0,0,3,0],
    [1,0,0,0,7,9,0,8,0],
    [0,0,4,0,0,0,0,0,9]
]

In [789]:
class expertparameters:
  def __init__(self):
    self.population = 500
    self.child_rate_per_generation = 12
    self.rate_of_gene_mutation = .1
    self.iterations = 600

In [790]:
expert_problem = problem(expert_puzzle)

In [793]:
expert_params = expertparameters()

In [794]:
bs_expert = run_genetic(expert_problem, expert_params)

Best Solution for iteration 0 has cost of 31
Best Solution for iteration 1 has cost of 28
Best Solution for iteration 2 has cost of 26
Best Solution for iteration 3 has cost of 26
Best Solution for iteration 4 has cost of 23
Best Solution for iteration 5 has cost of 20
Best Solution for iteration 6 has cost of 20
Best Solution for iteration 7 has cost of 18
Best Solution for iteration 8 has cost of 17
Best Solution for iteration 9 has cost of 15
Best Solution for iteration 10 has cost of 13
Best Solution for iteration 11 has cost of 12
Best Solution for iteration 12 has cost of 12
Best Solution for iteration 13 has cost of 11
Best Solution for iteration 14 has cost of 10
Best Solution for iteration 15 has cost of 10
Best Solution for iteration 16 has cost of 9
Best Solution for iteration 17 has cost of 9
Best Solution for iteration 18 has cost of 9
Best Solution for iteration 19 has cost of 9
Best Solution for iteration 20 has cost of 9
Best Solution for iteration 21 has cost of 9
Best

---
Expert Puzzle could not be solved after 10 minutes and 600 iterations

***Closest Solution Below***

---

In [795]:
bs_expert.chromosome

[[6, 8, 5, 2, 3, 4, 1, 9, 7],
 [9, 1, 3, 6, 5, 7, 8, 4, 2],
 [4, 2, 7, 9, 1, 8, 3, 5, 6],
 [5, 8, 1, 3, 4, 6, 9, 2, 7],
 [7, 6, 2, 8, 9, 5, 4, 1, 3],
 [3, 4, 9, 7, 2, 1, 6, 5, 8],
 [7, 9, 8, 4, 6, 2, 5, 3, 1],
 [1, 3, 6, 5, 7, 9, 2, 8, 4],
 [2, 5, 4, 1, 8, 3, 7, 6, 9]]

---

***MASTER PUZZLE***

Using a expert puzzle from Sudoku.com, we will try to solve it using our GA.

---

In [796]:
master_puzzle = [
    [9,0,6,0,0,8,0,5,1],
    [0,0,0,0,0,0,0,0,3],
    [8,0,0,2,0,0,0,0,0],
    [0,0,0,0,0,5,3,0,0],
    [0,0,9,0,0,0,2,0,0],
    [0,1,0,4,0,0,0,9,5],
    [0,4,0,0,7,0,0,0,0],
    [0,0,0,0,0,0,9,0,0],
    [0,0,1,0,0,2,0,6,8]
]

In [797]:
class masterparameters:
  def __init__(self):
    self.population = 500
    self.child_rate_per_generation = 15
    self.rate_of_gene_mutation = .1
    self.iterations = 800

In [798]:
master_problem = problem(master_puzzle)

In [799]:
master_params = masterparameters()

In [800]:
bs_master = run_genetic(master_problem, master_params)

Best Solution for iteration 0 has cost of 36
Best Solution for iteration 1 has cost of 30
Best Solution for iteration 2 has cost of 27
Best Solution for iteration 3 has cost of 26
Best Solution for iteration 4 has cost of 22
Best Solution for iteration 5 has cost of 21
Best Solution for iteration 6 has cost of 16
Best Solution for iteration 7 has cost of 15
Best Solution for iteration 8 has cost of 15
Best Solution for iteration 9 has cost of 13
Best Solution for iteration 10 has cost of 12
Best Solution for iteration 11 has cost of 12
Best Solution for iteration 12 has cost of 11
Best Solution for iteration 13 has cost of 11
Best Solution for iteration 14 has cost of 11
Best Solution for iteration 15 has cost of 11
Best Solution for iteration 16 has cost of 11
Best Solution for iteration 17 has cost of 11
Best Solution for iteration 18 has cost of 11
Best Solution for iteration 19 has cost of 11
Best Solution for iteration 20 has cost of 11
Best Solution for iteration 21 has cost of 1

---

Master Puzzle not solved after 18 minutes and 800 iterations

***Closest Solution Below***

---

In [801]:
bs_master.chromosome

[[9, 2, 6, 3, 4, 8, 7, 5, 1],
 [7, 1, 4, 5, 6, 9, 2, 8, 3],
 [8, 3, 5, 2, 1, 7, 6, 4, 9],
 [2, 6, 8, 7, 9, 5, 3, 1, 4],
 [4, 5, 9, 1, 8, 3, 2, 7, 6],
 [3, 1, 7, 4, 2, 6, 8, 9, 5],
 [6, 4, 3, 8, 7, 1, 5, 2, 9],
 [1, 8, 2, 6, 5, 4, 9, 3, 7],
 [5, 7, 1, 9, 3, 2, 4, 6, 8]]

---

***BLANK PUZZLE***

Given a blank puzzle, we will attempt to solve for any solution to the game

---

In [802]:
blank_puzzle = [
    [0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,0,0]
]

In [803]:
class blankparameters:
  def __init__(self):
    self.population = 500
    self.child_rate_per_generation = 10
    self.rate_of_gene_mutation = 0.1
    self.iterations = 500

In [804]:
blank_params = blankparameters()

In [805]:
blank_problem = problem(blank_puzzle)

In [806]:
bs_blank = run_genetic(blank_problem, blank_params)

Best Solution for iteration 0 has cost of 32
Best Solution for iteration 1 has cost of 32
Best Solution for iteration 2 has cost of 32
Best Solution for iteration 3 has cost of 30
Best Solution for iteration 4 has cost of 26
Best Solution for iteration 5 has cost of 26
Best Solution for iteration 6 has cost of 26
Best Solution for iteration 7 has cost of 26
Best Solution for iteration 8 has cost of 26
Best Solution for iteration 9 has cost of 26
Best Solution for iteration 10 has cost of 26
Best Solution for iteration 11 has cost of 26
Best Solution for iteration 12 has cost of 24
Best Solution for iteration 13 has cost of 24
Best Solution for iteration 14 has cost of 24
Best Solution for iteration 15 has cost of 24
Best Solution for iteration 16 has cost of 24
Best Solution for iteration 17 has cost of 24
Best Solution for iteration 18 has cost of 24
Best Solution for iteration 19 has cost of 23
Best Solution for iteration 20 has cost of 23
Best Solution for iteration 21 has cost of 2

---

The blank puzzle solution did not solve after 7 minutes and 500 iterations

***Closest solution below***

---

In [807]:
bs_blank.chromosome

[[1, 2, 3, 4, 5, 9, 8, 6, 7],
 [4, 5, 7, 2, 6, 8, 9, 1, 3],
 [8, 6, 9, 7, 3, 1, 5, 4, 2],
 [3, 7, 1, 4, 2, 5, 6, 9, 8],
 [6, 8, 2, 9, 7, 3, 4, 5, 1],
 [9, 4, 5, 8, 1, 6, 7, 2, 3],
 [2, 9, 6, 5, 3, 8, 1, 7, 4],
 [5, 1, 8, 6, 4, 7, 2, 3, 9],
 [7, 3, 4, 1, 9, 2, 8, 6, 5]]

## Results and conclusions    **(~10%)**

---

The results revealed a notable limitation in solving puzzles beyond the easy level. Despite its success in finding a solution for easy and medium-level puzzles, the algorithm faced increasing difficulty and inefficiency when confronted with harder puzzles. The algorithm encountered problems for puzzles categorised as hard, expert, master and blank. The algorithm failed to converge to a solution within a reasonable time frame. The GA seems to get stuck on a certain cost for a prolonged period of time without improvement. This outcome underscores the inherent complexity of Sudoku puzzles beyond the easy difficulty level. It is important to acknowledge that without limitations on the algorithm's iterations, there exists the potential for solving more challenging puzzles, albeit at the cost of significantly increased computation time. However, it is essential to highlight that the algorithm's performance in solving harder puzzles is currently hampered by its relatively slow solution, indicating the need for further optimisation or alternative strategies to enhance its efficacy in tackling more intricate Sudoku configurations.

---

